# Analysis of Detached Housing Sales in Western Prince William County, Virginia from 1/1/2010 - 3/30/2021


## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook')
sns.set_style('white')

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import TransformedTargetRegressor, make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import RepeatedKFold, cross_val_score

## Import Data

In [2]:
df = pd.read_csv('data/detached_sales.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52459 entries, 0 to 52458
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   MLS #                      52459 non-null  object 
 1   Address                    52459 non-null  object 
 2   Zip Code                   52459 non-null  int64  
 3   City                       52459 non-null  object 
 4   DOM                        52459 non-null  object 
 5   Close Date                 52459 non-null  object 
 6   Close Price                52459 non-null  object 
 7   Beds                       52459 non-null  int64  
 8   Bathrooms Full             52459 non-null  int64  
 9   Bathrooms Half             47393 non-null  float64
 10  Subdivision/Neighborhood   52458 non-null  object 
 11  Structure Type             52459 non-null  object 
 12  Basement                   52459 non-null  object 
 13  Fireplaces Total           51286 non-null  flo

In [4]:
df.isnull().sum()

MLS #                            0
Address                          0
Zip Code                         0
City                             0
DOM                              0
Close Date                       0
Close Price                      0
Beds                             0
Bathrooms Full                   0
Bathrooms Half                5066
Subdivision/Neighborhood         1
Structure Type                   0
Basement                         0
Fireplaces Total              1173
Levels/Stories                  65
Total Garage Spaces          12412
Above Grade Finished SQFT    22529
Below Grade Finished SQFT    30867
Condo/Coop Assoc YN              3
Fireplace                     1705
Garage                           0
HOA YN                           0
List Price                       0
Lot Size SqFt                   84
School District              31924
Total Finished SQFT          22464
Year Built                      14
dtype: int64

## Data Cleaning

### Column Headers

In [ ]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('/', '_')
df.columns = df.columns.str.lower()

### close_price

In [ ]:
df['close_price'] = df.close_price.str.replace('$', '', regex=False)
df['close_price'] = df.close_price.str.replace(',', '')
df['close_price'] = df.close_price.str.split('.', 1, expand=True)[0].astype(int)

### check for odd closing prices at the low end

In [ ]:
df.loc[df.close_price <= 10_000, :]

MLS # VAPW273384 looks as if the agent did not input the correct closing price, so the close_price will be changed to $322,000 from 2200.

MLS # 1002244130 looks as if the agent did not add the trailing two 0s to the sale documentation, so the close_price will be changed to $431,000


In [ ]:
df['close_price'] = np.where(df.close_price == 2200, 322000, df.close_price)
df['close_price'] = np.where(df.close_price == 4310, 431000, df.close_price)


### check for odd closing prices at the high end

In [ ]:
df.loc[df.close_price >= 1_500_000, :]

### Plot of close_price distribution

In [ ]:
fig1, ax0 = plt.subplots(1, 1, figsize=(16, 7))
fig1.suptitle('Distribution of Detached Home Sales close_price, 1/1/2000 - 6/15/2021')

sns.histplot(data=df,
             x=df.close_price,
             bins= 200,
             kde=True,
             ax = ax0)

ax0.set_xlabel('Closing Price ($M)')
ax0.set_ylabel('Closing Price Frequency')

### Plot comparing original close_price and log-transformed close_price distributions

In [ ]:
fig2, (ax0, ax1) = plt.subplots(1, 2, figsize=(16, 7))
fig2.suptitle('closing_price Distributions')

sns.histplot(data=df,
             x=df.close_price,
             bins= 200,
             kde=True,
             ax = ax0)
ax0.set_xlabel('Closing Price ($M)')
ax0.set_ylabel('Closing Price Frequency')

sns.histplot(data=df,
             x=np.log(df['close_price']),
             bins= 200,
             kde=True,
             ax = ax1)
ax1.set_xlabel('Log Closing Price (log($M))')
ax1.set_ylabel('Closing Price Frequency')

### close_date

In [ ]:
df['close_date'] = pd.to_datetime(df.close_date)
df['close_month'] = pd.DatetimeIndex(df.close_date).month
df['close_year'] = pd.DatetimeIndex(df.close_date).year

In [ ]:
np.max(df.close_date)

In [ ]:
df.loc[df.close_date >= '2021-06-16', :]

### One house's closing date is listed beyond the dataset
#### As of 6/16/2021, the new, updated dataset does not include house closings beyond the dataset end date 6/15/2021
Instead of the closing date from VAPW517178 being 6/25, it is getting reassigned to 3/25

In [ ]:
# df.loc[df['mls_#'] == 'VAPW517178', :]

In [ ]:
# df['close_date'] = df['close_date'].replace(pd.to_datetime('2021-06-25'),
#                                             pd.to_datetime('2021-03-25'))

In [ ]:
# df.loc[df['mls_#'] == 'VAPW517178', :]

### mls_num

In [ ]:
df.loc[df['mls_#'].duplicated(keep=False), :]


In [ ]:
# df.drop_duplicates(subset='mls_#',
#                    keep='first',
#                    ignore_index=True,
#                    inplace=True)


### zip_code

In [ ]:
df['zip_code'] = df.zip_code.astype('str')

In [ ]:
print(type(df.zip_code[1]))
df.zip_code.dtypes


In [ ]:
df.zip_code.value_counts()

#### Update Zip Codes

Three zip codes are contained wholly within other zip codes. These three zip codes account for only 23 detached sales for the time period covered in this analysis. These three zip codes are being recoded to the larger zip code within which they reside.

20108 -->> 20110

20156 -->> 20155

20168 -->> 20169

In [ ]:
df['zip_code'] = np.where(df.zip_code == '20108', '20110', df.zip_code)
df['zip_code'] = np.where(df.zip_code == '20156', '20155', df.zip_code)
df['zip_code'] = np.where(df.zip_code == '20168', '20169', df.zip_code)

In [ ]:
print(df.zip_code.dtypes)
df.zip_code.value_counts()

In [ ]:
fig3, ax0 = plt.subplots(1, 1, figsize=(16, 7))

sns.scatterplot(data=df.loc[df['zip_code'] == '20155', :],
                x='close_date',
                y='close_price',
                alpha=0.05,
                ax=ax0)
plt.axhline(y=575000, ls='--', c='red')
ax0.set_title('Detached Closing Prices, 2000 - 2021 (partial)')
ax0.set_xlabel('Year')
ax0.set_ylabel('Closing Price ($)')

In [ ]:
fig4, ax0 = plt.subplots(1, 1, figsize=(16, 7))

sns.boxplot(data=df,
            x='close_year',
            y='close_price',
            color='lavender',
            notch=True,
            ax=ax0)
ax0.set_title('Detached Closing Prices, 2000 - 2021 (partial)')
ax0.set_xlabel('Year')
ax0.set_ylabel('Closing Price ($M)')

In [ ]:
sns.relplot(data=df,
            x='close_date',
            y='close_price',
            kind='scatter',
            col='zip_code',
            col_wrap=3,
            alpha=0.08)

#### DOM

In [ ]:
df['dom'] = df.dom.str.replace(',', '', regex=False)

In [ ]:
df['dom'] = df.dom.astype(int)

In [ ]:
pd.DataFrame(df.groupby(['close_month'])['dom'].describe())

In [ ]:
fig5, ax0 = plt.subplots(1, 1, figsize=(16, 7))

sns.boxplot(data=df,
            x='close_month',
            y='dom',
            color='lavender',
            notch=True,
            showmeans=True,
            showfliers=False,
            ax=ax0)

In [ ]:
fig6, ax0 = plt.subplots(1, 1, figsize=(16, 7))

sns.scatterplot(data=df,
                x='close_date',
                y='dom',
                alpha=0.06,
                ax=ax0)

In [ ]:
fig7, ax0 = plt.subplots(1, 1, figsize=(16, 7))

sns.scatterplot(data=df,
                x='close_price',
                y='dom',
                alpha=0.06,
                ax=ax0)

#### HOA

In [ ]:
df.rename(columns={'hoa_yn': 'hoa'}, inplace=True)

In [ ]:
hoa_map = {'Yes': 1,
           'No': 0}

df['hoa'] = df.hoa.replace(hoa_map).astype('int')

In [ ]:
df.hoa.value_counts(dropna=False)

#### Garage

In [ ]:
garage_map = {'Yes': 1,
              'No': 0}

In [ ]:
df['garage'] = df.garage.replace(garage_map).astype('int')

In [ ]:
df.garage.value_counts(dropna=False)

### Bathrooms - full

In [ ]:
df.bathrooms_full.value_counts(dropna=False)

Several properties have likely erroneous numbers of full bathrooms, for those listings that have more than 7 full bathrooms, the house is being dropped from the dataset

In [ ]:
df.loc[df.bathrooms_full >= 8, :]

In [ ]:
df = df.loc[df.bathrooms_full <8, :]

In [ ]:
df.bathrooms_full.value_counts(dropna=False)

### Bathrooms - half

In [ ]:
df.bathrooms_half.value_counts(dropna=False)

Approximately 1400 listings have null values for half bathrooms. Those listings will be dropped from the dataset

In [ ]:
df.loc[df.bathrooms_half.notnull(), :]

In [ ]:
df = df.loc[df.bathrooms_half.notnull(), :]

In [ ]:
df.bathrooms_half.value_counts(dropna=False)

#### Fireplaces_total

In [ ]:
df.fireplaces_total.value_counts(dropna=False)

Approximately 560 listings have null values for total number of fireplaces. Those listings will be dropped from the dataset

In [ ]:
df.loc[df.fireplaces_total.notnull(), :]

In [ ]:
df = df.loc[df.fireplaces_total.notnull(), :]

In [ ]:
df.fireplaces_total.value_counts(dropna=False)

In [ ]:
df.isnull().sum()

#### Condo_Coop_Assoc_YN

In [ ]:
df.rename(columns={'condo_coop_assoc_yn': 'condo_coop_assoc'}, inplace=True)

In [ ]:
df.condo_coop_assoc.value_counts(dropna=False)

One listing is a null value for condo_coop_assoc. Listing will be dropped from dataset

In [ ]:
df.loc[df.condo_coop_assoc.notnull(), :]

In [ ]:
df = df.loc[df.condo_coop_assoc.notnull(), :]

In [ ]:
df.condo_coop_assoc.value_counts(dropna=False)

In [ ]:
condo_coop_map = {'Yes': 1,
                  'No': 0}

In [ ]:
df['condo_coop_assoc'] = df['condo_coop_assoc'].replace(condo_coop_map)

In [ ]:
df.condo_coop_assoc.value_counts(dropna=False)

In [ ]:
df.isnull().sum()

#### Basement

In [ ]:
basement_map = {'Yes': 1,
                'No': 0}

In [ ]:
df['basement'] = df.basement.replace(basement_map)

In [ ]:
df.basement.value_counts(dropna=False)

### Levels_stories

In [ ]:
df.levels_stories.value_counts(dropna=False)

Drop null listings

In [ ]:
df.loc[df.levels_stories.notnull(), :]

In [ ]:
df = df.loc[df.levels_stories.notnull(), :]

In [ ]:
df.levels_stories.value_counts(dropna=False)

In [ ]:
levels_stories_map = {1: 1,
                      1.5: 1.5,
                      2: 2,
                      2.5: 2.5,
                      3: 3,
                      3.5: 3,
                      '3+': 3,
                      4: 3,
                      5: 3,
                      'Other': 3}

In [ ]:
df['levels_stories'] = df.levels_stories.replace(levels_stories_map).astype(float)

In [ ]:
df.levels_stories.value_counts(dropna=False)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
print(df.isnull().sum())
print(df.info())
print(df.head(5))

### Correlation Heatmap of correlation matrix

In [ ]:
df_corr = df.corr()

In [ ]:
mask = np.triu(np.ones_like(df_corr, dtype=bool))

fig6, ax0 = plt.subplots(1, 1, figsize=(16, 7))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(df_corr,
            mask=mask,
            cmap=cmap,
            vmax=1,
            center=0,
            vmin=-1,
            square=True,
            linewidths=0.5,
            cbar_kws={'shrink': 0.5})

### Create X and y datasets

In [ ]:
df.info()

In [ ]:
df.iloc[:, np.r_[5, 26:28]]

In [ ]:
df = df.iloc[:, np.r_[5, 26:28]]

In [ ]:
print(df.info())
df.describe()

In [ ]:
X = df.iloc[:, np.r_[1:3]]
y = df['close_price']

In [ ]:
X_features = list(X.columns)
X_features

### Split data into training and test sets


In [ ]:
random_seed = 0
train_pct = 0.7
test_pct = 0.3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=test_pct,
                                                    train_size=train_pct,
                                                    random_state=random_seed)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


## Pipeline Objects


#### Numerical and Categorical Column Lists

In [ ]:
num_columns = make_column_selector(dtype_include=np.number)
print(num_columns(X_train))

In [ ]:
cat_columns = make_column_selector(dtype_include=[object, 'category'])
print(cat_columns(X_train))

### Preprocessing

#### Numerical Transformers

In [ ]:
num_tree_col_processor = SimpleImputer(strategy='median')
num_non_tree_col_processor = StandardScaler()

In [ ]:
num_non_tree_col_processor.fit_transform(X_train)

#### Categorical Transformers

In [ ]:
cat_tree_col_processor = OneHotEncoder(handle_unknown='ignore')
cat_non_tree_col_processor = OneHotEncoder(handle_unknown='ignore')

#### Create Tree and Non-tree Column Preprocessors

In [ ]:
tree_preprocessor = make_column_transformer((cat_tree_col_processor, cat_columns),
                                            (num_tree_col_processor, num_columns),
                                            remainder='passthrough')
tree_preprocessor

In [ ]:
non_tree_preprocessor = make_column_transformer((cat_non_tree_col_processor, cat_columns),
                                                (num_non_tree_col_processor, num_columns),
                                                remainder='passthrough')
non_tree_preprocessor

### Cross validation object

In [ ]:
rkf_cv = RepeatedKFold(n_splits=5,
                       n_repeats=5,
                       random_state=random_seed)

rkf_gs = RepeatedKFold(n_splits=5,
                       n_repeats=5,
                       random_state=random_seed)

### Specify models

In [ ]:
lin_reg = LinearRegression()
ridge_reg = Ridge()
rf_reg = RandomForestRegressor()
knn_reg =KNeighborsRegressor()

### Linear Regression

In [ ]:
lin_reg_pipe = make_pipeline(non_tree_preprocessor,
                             TransformedTargetRegressor(
                                 regressor=lin_reg,
                                 func=np.log,
                                 inverse_func=np.exp))

In [ ]:
lin_reg_best_model = lin_reg_pipe.fit(X_train, y_train)

In [ ]:
print('Linear model intercept:')
lin_reg_best_model.named_steps['transformedtargetregressor'].regressor_.intercept_

In [ ]:
print('Linear model feature coefficients:')
lin_reg_best_model.named_steps['transformedtargetregressor'].regressor_.coef_

In [ ]:
# https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html
# https://stackoverflow.com/questions/43856280/return-coefficients-from-pipeline-object-in-sklearn
# lin_reg_pipe.named_steps['linearregression'].coef_
# lin_reg_pipe['linearregression'].coef_
# lin_reg_pipe['linearregression'].intercept_

In [ ]:
pd.DataFrame({'Feature': X_train.columns,
              'Coefficients': np.transpose(lin_reg_pipe.named_steps['transformedtargetregressor'].regressor_.coef_)})

In [ ]:
lin_reg_pipe_scores = cross_val_score(lin_reg_pipe,
                                      X_train,
                                      y_train,
                                      cv=rkf_cv,
                                      verbose=True,
                                      scoring='neg_mean_squared_error') * -1

In [ ]:
lin_reg_pipe_scores.mean(), lin_reg_pipe_scores.std()

In [ ]:
lin_reg_best_model_preds = lin_reg_best_model.predict(X_test)

In [ ]:
lin_reg_best_model_r2 = r2_score(y_test, lin_reg_best_model_preds)

In [ ]:
lin_reg_best_model_r2

In [ ]:
lin_reg_best_model_mse = mean_squared_error(y_test, lin_reg_best_model_preds)

In [ ]:
lin_reg_best_model_mse

In [ ]:
np.sqrt(lin_reg_best_model_mse)

In [ ]:
fig5, ax0 = plt.subplots(1, 1, figsize=(16, 7))

sns.scatterplot(x = y_test,
                y = lin_reg_best_model_preds,
                alpha=0.2)
ax0.set_title('Linear Regression Pipeline Performance')
ax0.set_xlabel('Actual Values')
ax0.set_ylabel('Linear Regression Pipeline Predicted Values')

In [ ]:
ridge_reg_pipe = make_pipeline(non_tree_preprocessor,
                               ridge_reg)

In [ ]:
rf_reg_pipe = make_pipeline(tree_preprocessor,
                            rf_reg)


In [ ]:
knn_reg_pipe = make_pipeline(tree_preprocessor,
                             knn_reg)

### Evaluate Pipelines

In [ ]:
lin_reg_pipe_score = cross_val_score(lin_reg_pipe, X_train, y_train, cv=rkf_cv, scoring='neg_mean_squared_error')

In [ ]:
ridge_reg_pipe_score = cross_val_score(ridge_reg_pipe, X_train, y_train, cv=rkf_cv, scoring='neg_mean_squared_error')

In [ ]:
rf_reg_pipe_score = cross_val_score(rf_reg_pipe, X_train, y_train, cv=rkf_cv, scoring='neg_mean_squared_error')

In [ ]:
knn_reg_pipe_score = cross_val_score(knn_reg_pipe, X_train, y_train, cv=rkf_cv, scoring='neg_mean_squared_error')

In [ ]:
print(f'Linear Regression MSE: {lin_reg_pipe_score.mean()}')
print(f'Ridge Regression MSE: {ridge_reg_pipe_score.mean()}')
print(f'Random Forest Regression MSE: {rf_reg_pipe_score.mean()}')
print(f'KNN Regression MSE: {knn_reg_pipe_score.mean()}')

### Fit pipelines with training data

In [ ]:
lin_reg_pipe.fit(X_train, y_train)

In [ ]:
ridge_reg_pipe.fit(X_train, y_train)

In [ ]:
rf_reg_pipe.fit(X_train, y_train)

In [ ]:
knn_reg_pipe.fit(X_train, y_train)

### Use test data on pipeline to predict

In [ ]:
lin_reg_pipe.predict(X_test)

In [ ]:
ridge_reg_pipe.predict(X_test)

In [ ]:
rf_reg_pipe.predict(X_test)

In [ ]:
knn_reg_pipe.predict(X_test)

### Prediction results